## Домашнее задание
Взять boston house-prices datase (sklearn.datasets.load_boston) и сделать тоже самое для задачи регрессии (попробовать разные алгоритмы, поподбирать параметры, вывести итоговое качество)

In [1]:
import pandas as pd
import numpy as np

from sklearn.datasets import load_boston

from sklearn import cross_validation

from sklearn.model_selection import GridSearchCV

from sklearn import linear_model

from sklearn.metrics import mean_absolute_error, mean_squared_error, median_absolute_error, r2_score

from sklearn.preprocessing import StandardScaler

#https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter

C:\Program Files\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
boston = load_boston()
X = pd.DataFrame(boston.data, columns=boston.feature_names)
y = pd.DataFrame(boston.target, columns=["target"])

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)


In [3]:
X.info(), y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 13 columns):
CRIM       506 non-null float64
ZN         506 non-null float64
INDUS      506 non-null float64
CHAS       506 non-null float64
NOX        506 non-null float64
RM         506 non-null float64
AGE        506 non-null float64
DIS        506 non-null float64
RAD        506 non-null float64
TAX        506 non-null float64
PTRATIO    506 non-null float64
B          506 non-null float64
LSTAT      506 non-null float64
dtypes: float64(13)
memory usage: 51.5 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 1 columns):
target    506 non-null float64
dtypes: float64(1)
memory usage: 4.0 KB


(None, None)

In [4]:
X.describe(), y.describe()

(             CRIM          ZN       INDUS        CHAS         NOX          RM  \
 count  506.000000  506.000000  506.000000  506.000000  506.000000  506.000000   
 mean     3.593761   11.363636   11.136779    0.069170    0.554695    6.284634   
 std      8.596783   23.322453    6.860353    0.253994    0.115878    0.702617   
 min      0.006320    0.000000    0.460000    0.000000    0.385000    3.561000   
 25%      0.082045    0.000000    5.190000    0.000000    0.449000    5.885500   
 50%      0.256510    0.000000    9.690000    0.000000    0.538000    6.208500   
 75%      3.647423   12.500000   18.100000    0.000000    0.624000    6.623500   
 max     88.976200  100.000000   27.740000    1.000000    0.871000    8.780000   
 
               AGE         DIS         RAD         TAX     PTRATIO           B  \
 count  506.000000  506.000000  506.000000  506.000000  506.000000  506.000000   
 mean    68.574901    3.795043    9.549407  408.237154   18.455534  356.674032   
 std     28.14

### Создаю модели. Сразу задаю, где возможно, параллельные вычисления

#### Линейная регрессия

In [5]:
from sklearn.linear_model import LinearRegression

model_LR = LinearRegression(n_jobs = -1)
model_LR.get_params()

{'copy_X': True, 'fit_intercept': True, 'n_jobs': -1, 'normalize': False}

In [6]:
model_LR.fit(x_train, y_train)

y_train_pred = model_LR.predict(x_train)
y_test_pred = model_LR.predict(x_test)

In [7]:
#Оченка качества на дефолте
print(model_LR.coef_)

print('MSE train: {:.3f}, test: {:.3f}'.format(mean_squared_error(y_train, y_train_pred),mean_squared_error(y_test, y_test_pred)))

print('R^2 train: {:.3f}, test: {:.3f}'.format(r2_score(y_train, y_train_pred),r2_score(y_test, y_test_pred)))

[[-5.06231893e-02  4.83255430e-02 -2.68292126e-02  2.68031364e+00
  -1.70907879e+01  3.80610717e+00  5.16928598e-03 -1.48514953e+00
   2.40420364e-01 -8.95052139e-03 -1.00201805e+00  1.04903456e-02
  -5.19851329e-01]]
MSE train: 19.982, test: 30.674
R^2 train: 0.755, test: 0.680


In [8]:
#Делю данные, создаю словарь параметров и пдбираю лучшие параметры

x_train, x_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size = 0.3, random_state = 0)

param_grid_LR = {
    'copy_X': ['True', 'False'],
    'fit_intercept': ['True', 'False'],
    'normalize': ['True', 'False']
}

grid_LR = GridSearchCV(model_LR, param_grid_LR, n_jobs=-1, scoring ='r2', cv = 5)
grid_LR.fit(X,y)

grid_LR.best_params_

{'copy_X': 'True', 'fit_intercept': 'True', 'normalize': 'True'}

In [9]:
grid_LR.best_estimator_

LinearRegression(copy_X='True', fit_intercept='True', n_jobs=-1,
         normalize='True')

In [10]:
#Оцениваю качество на лучших параметрах

grid_LR.best_estimator_.fit(x_train, y_train)

y_train_pred_best = grid_LR.best_estimator_.predict(x_train)
y_test_pred_best = grid_LR.best_estimator_.predict(x_test)

#Оченка качества
print('MSE train: {:.3f}, test: {:.3f}'.format(mean_squared_error(y_train, y_train_pred_best),mean_squared_error(y_test, y_test_pred_best)))
print('R^2 train: {:.3f}, test: {:.3f}'.format(r2_score(y_train, y_train_pred_best),r2_score(y_test, y_test_pred_best)))

MSE train: 19.966, test: 27.184
R^2 train: 0.764, test: 0.674


#### Лассо регрессия

In [11]:
from sklearn.linear_model import Lasso

model_Lasso = Lasso(alpha = 0.1)
model_Lasso.get_params()

{'alpha': 0.1,
 'copy_X': True,
 'fit_intercept': True,
 'max_iter': 1000,
 'normalize': False,
 'positive': False,
 'precompute': False,
 'random_state': None,
 'selection': 'cyclic',
 'tol': 0.0001,
 'warm_start': False}

In [12]:
#Подготавливаю данные - шкалирую

from sklearn.preprocessing import StandardScaler

sc_x = StandardScaler()
sc_y = StandardScaler()

X_std = sc_x.fit_transform(X)
y_std = sc_y.fit_transform(y).flatten()

x_train_scaled, x_test_scaled, y_train_scaled, y_test_scaled = cross_validation.train_test_split(X_std, y_std, test_size = 0.3, random_state = 0)

x_train_scaled.std(), x_train_scaled.mean()

(0.9936823753348005, -0.0023455983746008984)

In [13]:
#обучаю модель, получаю прогноз и оцениваю качетво на дефолте
model_Lasso.fit(x_train_scaled, y_train_scaled)

y_train_pred = model_Lasso.predict(x_train_scaled)
y_test_pred = model_Lasso.predict(x_test_scaled)

print(model_Lasso.coef_)

print('MSE train: {:.3f}, test: {:.3f}'.format(mean_squared_error(y_train_scaled, y_train_pred),
        mean_squared_error(y_test_scaled, y_test_pred)))

print('R^2 train: {:.3f}, test: {:.3f}'.format(r2_score(y_train_scaled, y_train_pred),r2_score(y_test_scaled, y_test_pred)))

[-0.00155798  0.         -0.          0.         -0.          0.29618857
 -0.         -0.         -0.         -0.01976187 -0.19626599  0.
 -0.37360559]
MSE train: 0.305, test: 0.395
R^2 train: 0.696, test: 0.600


In [14]:
#Делю данные, создаю словарь параметров и подбираю лучшие параметры

x_train_scaled, x_test_scaled, y_train_scaled, y_test_scaled = cross_validation.train_test_split(X_std, y_std, test_size = 0.3, random_state = 0)

param_grid_Lasso = {
    'alpha': np.linspace(0.0001, 0.1, num = 10),
    'fit_intercept': ['True', 'False'],
    'max_iter': np.linspace(50, 200, num = 10),
    'normalize': ['True', 'False'],
    'tol': np.linspace(0.0001, 0.01, num = 10)
}

grid_Lasso = GridSearchCV(model_Lasso, param_grid_Lasso, n_jobs = -1, scoring ='r2', cv = 5)
grid_Lasso.fit(X,y)

grid_Lasso.best_params_

{'alpha': 0.0112,
 'fit_intercept': 'True',
 'max_iter': 50.0,
 'normalize': 'True',
 'tol': 0.0012000000000000001}

In [15]:
#Оцениваю качество на лучших параметрах
grid_Lasso.best_estimator_.fit(x_train_scaled, y_train_scaled)

y_train_pred_best = grid_Lasso.best_estimator_.predict(x_train_scaled)
y_test_pred_best = grid_Lasso.best_estimator_.predict(x_test_scaled)

#Оченка качества
print('MSE train: {:.3f}, test: {:.3f}'.format(mean_squared_error(y_train_scaled, y_train_pred_best),
                                               mean_squared_error(y_test_scaled, y_test_pred_best)))

print('R^2 train: {:.3f}, test: {:.3f}'.format(r2_score(y_train_scaled, y_train_pred_best),
                                               r2_score(y_test_scaled, y_test_pred_best)))

MSE train: 0.361, test: 0.423
R^2 train: 0.640, test: 0.571


#### ElasticNet

In [16]:
from sklearn.linear_model import ElasticNet

model_EN = ElasticNet(alpha = 0.1)
model_EN.get_params()

{'alpha': 0.1,
 'copy_X': True,
 'fit_intercept': True,
 'l1_ratio': 0.5,
 'max_iter': 1000,
 'normalize': False,
 'positive': False,
 'precompute': False,
 'random_state': None,
 'selection': 'cyclic',
 'tol': 0.0001,
 'warm_start': False}

In [17]:
#Подготавливаю данные - шкалирую

sc_x = StandardScaler()
sc_y = StandardScaler()

X_std = sc_x.fit_transform(X)
y_std = sc_y.fit_transform(y).flatten()

x_train_scaled, x_test_scaled, y_train_scaled, y_test_scaled = cross_validation.train_test_split(X_std, y_std, test_size = 0.3, random_state = 0)

x_train_scaled.std(), x_train_scaled.mean()

(0.9936823753348005, -0.0023455983746008984)

In [18]:
#обучаю модель, получаю прогноз и оцениваю качетво на дефолте

model_EN.fit(x_train_scaled, y_train_scaled)

y_train_pred = model_EN.predict(x_train_scaled)
y_test_pred = model_EN.predict(x_test_scaled)

print(model_EN.coef_)

print('MSE train: {:.3f}, test: {:.3f}'.format(mean_squared_error(y_train_scaled, y_train_pred),
        mean_squared_error(y_test_scaled, y_test_pred)))

print('R^2 train: {:.3f}, test: {:.3f}'.format(r2_score(y_train_scaled, y_train_pred),r2_score(y_test_scaled, y_test_pred)))

[-0.03530421  0.         -0.          0.04029752 -0.01646424  0.31846627
 -0.         -0.02019854 -0.         -0.03136783 -0.21312727  0.03627586
 -0.35100948]
MSE train: 0.280, test: 0.378
R^2 train: 0.721, test: 0.617


In [19]:
#Делю данные, создаю словарь параметров и подбираю лучшие параметры

x_train_scaled, x_test_scaled, y_train_scaled, y_test_scaled = cross_validation.train_test_split(X_std, y_std, test_size = 0.3, random_state = 0)

param_grid_EN = {
    'alpha': np.linspace(0.00001, 0.001, num = 10),
    'fit_intercept': ['True', 'False'],
    'l1_ratio': np.linspace(0.000001, 0.00001, num = 10),
    'max_iter': np.linspace(100, 200, num = 5),
    'normalize': ['True', 'False'],
}

grid_EN = GridSearchCV(model_EN, param_grid_EN, scoring ='r2', n_jobs = -1, cv = 5)
grid_EN.fit(X,y)

grid_EN.best_params_

{'alpha': 0.0007800000000000001,
 'fit_intercept': 'True',
 'l1_ratio': 1e-06,
 'max_iter': 100.0,
 'normalize': 'True'}

In [20]:
#Оцениваю качество на лучших параметрах
grid_EN.best_estimator_.fit(x_train_scaled, y_train_scaled)

y_train_pred_best = grid_EN.best_estimator_.predict(x_train_scaled)
y_test_pred_best = grid_EN.best_estimator_.predict(x_test_scaled)

#Оченка качества
print('MSE train: {:.3f}, test: {:.3f}'.format(mean_squared_error(y_train_scaled, y_train_pred_best),
                                               mean_squared_error(y_test_scaled, y_test_pred_best)))

print('R^2 train: {:.3f}, test: {:.3f}'.format(r2_score(y_train_scaled, y_train_pred_best),
                                               r2_score(y_test_scaled, y_test_pred_best)))

MSE train: 0.262, test: 0.362
R^2 train: 0.739, test: 0.633


#### Регрессия Bayesian

In [21]:
from sklearn.linear_model import BayesianRidge

model_ByR = BayesianRidge()
model_ByR.get_params()

{'alpha_1': 1e-06,
 'alpha_2': 1e-06,
 'compute_score': False,
 'copy_X': True,
 'fit_intercept': True,
 'lambda_1': 1e-06,
 'lambda_2': 1e-06,
 'n_iter': 300,
 'normalize': False,
 'tol': 0.001,
 'verbose': False}

In [22]:
#Создаю модель
model_ByR.fit(x_train, y_train)

y_train_pred = model_ByR.predict(x_train)
y_test_pred = model_ByR.predict(x_test)

C:\Program Files\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [23]:
#Оченка качества на дефолте
print(model_ByR.coef_)

print('MSE train: {:.3f}, test: {:.3f}'.format(mean_squared_error(y_train, y_train_pred),mean_squared_error(y_test, y_test_pred)))

print('R^2 train: {:.3f}, test: {:.3f}'.format(r2_score(y_train, y_train_pred),r2_score(y_test, y_test_pred)))

[-0.11424918  0.04893541 -0.0479451   1.65085687 -1.24857157  3.52540128
 -0.02100559 -1.30394739  0.21306009 -0.0128196  -0.87827693  0.00783844
 -0.53236183]
MSE train: 20.740, test: 28.630
R^2 train: 0.755, test: 0.656


In [24]:
#Делю данные, создаю словарь параметров и пдбираю лучшие параметры

x_train, x_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size = 0.3, random_state = 0)

param_grid_ByR = {
    'alpha_1': np.linspace(1e-07, 1e-06, num = 10),
    'lambda_1': np.linspace(1e-07, 1e-06, num = 10),
    }

grid_ByR = GridSearchCV(model_ByR, param_grid_ByR, n_jobs=-1, scoring ='r2',  cv = 5)
grid_ByR.fit(X,y)

grid_ByR.best_params_

C:\Program Files\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


{'alpha_1': 1e-06, 'lambda_1': 1e-07}

In [25]:
#Оцениваю качество на лучших параметрах
grid_ByR.best_estimator_.fit(x_train_scaled, y_train_scaled)

y_train_pred_best = grid_ByR.best_estimator_.predict(x_train_scaled)
y_test_pred_best = grid_ByR.best_estimator_.predict(x_test_scaled)

#Оченка качества
print('MSE train: {:.3f}, test: {:.3f}'.format(mean_squared_error(y_train_scaled, y_train_pred_best),
                                               mean_squared_error(y_test_scaled, y_test_pred_best)))

print('R^2 train: {:.3f}, test: {:.3f}'.format(r2_score(y_train_scaled, y_train_pred_best),
                                               r2_score(y_test_scaled, y_test_pred_best)))

MSE train: 0.237, test: 0.326
R^2 train: 0.764, test: 0.670
